In [1]:
from transrepair_kor.main import Sentence, test_consistency, repair, calc_consistency_score, Sentence
from tqdm import tqdm
import pandas as pd


In [2]:
df = pd.read_csv('./data/groups_kor_eng/gender.csv')

kor_sentences = []
eng_sentences = []

for i, row in df.iterrows():
    kor_sentences.append(row["원문"])
    eng_sentences.append(row["번역문"])

In [3]:
len(kor_sentences)

160399

In [4]:
# text = Sentence("마지막으로 그는 자신이 아끼는 엽서에 할아버지를 위해 그림을 그려줘요.", debug=True)
sentences = {}
for s in tqdm(kor_sentences[:1000]):
    text = Sentence(s)
    sentences[text.hash] = text


100%|██████████| 1000/1000 [07:23<00:00,  2.25it/s]


In [5]:
mutant_sentences = {}

In [6]:
for hash, text in tqdm(sentences.items()):
    if hash in mutant_sentences:
        continue
    mutants = text.create_mutations()
    mutant_sentences[hash] = mutants

100%|██████████| 883/883 [1:00:32<00:00,  4.11s/it]


In [15]:
import pickle

with open('kor_parallel_sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

with open('kor_parallel_mutant_sentences.pkl', 'wb') as f:
    pickle.dump(mutant_sentences, f)

In [1]:
from transrepair_kor.main import Sentence, test_consistency, repair, calc_consistency_score
from tqdm import tqdm
import pandas as pd
import pickle

2022-06-06 21:54:38,133 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [2]:
with open('kor_parallel_sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

with open('kor_parallel_mutant_sentences.pkl', 'rb') as f:
    mutant_sentences = pickle.load(f)

In [3]:
repair_results = []
inconsistency_results = []

for hash, text in tqdm(sentences.items()):
    mutants = mutant_sentences[hash]
    is_consistent, (inconsistent_mutant, score) = test_consistency(text, mutants)
    if not is_consistent:
        repaired_sentence = repair(text, mutants)
        inconsistency_results.append((inconsistent_mutant, score))
        repair_results.append((text, mutants, repaired_sentence))

100%|██████████| 883/883 [01:55<00:00,  7.63it/s]


In [4]:
for text, mutants, repaired_sentence in repair_results:
    if text.translated != repaired_sentence.translated:
        print(f"{text.sentence}")
        print(f"{text.translated}")
        print(f"{repaired_sentence.mutant_translated} => {repaired_sentence.translated} ({repaired_sentence.src_aligned} => {repaired_sentence.dest_aligned})")
        print("")

11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.
In chapter 11, Jesus calls Lazarus this time from the tomb and raises him from the dead.
At the oxen, this time Jesus called Lazarus out of the tomb and raised him from the dead. => At the chapter this time Jesus called Lazarus out of the tomb and raised him from the dead. (oxen, => chapter)

A는 오늘 예방접종을 하러 병원을 갔는데, 저 때까지 주사 맞는지 모르고 기분이 엄청 좋았어요.
A went to the hospital to get vaccinated today, and until then, she didn't know if she was getting an injection, and she felt so good.
A went to the hospital for a treatment vaccination today, and he did not know he was getting the injection until then, and he felt so good. => A went to the hospital for a get vaccinated today, and he did not know he was getting the injection until then, and he felt so good. (treatment vaccination => get vaccinated)

c는 결혼은 자신의 삶을 경제적으로 안정되게 할 수단이라고 생각해요.
c thinks that marriage is a means of making one's life economically stable.
c thinks that living together is a

In [ ]:
len(repair_results)

247